In [1]:
from mongoconnection import *
import pandas as pd

In [2]:
dat = pd.read_csv('../data/cbd_info2.csv')

In [3]:
def lista ():
    a = dat.Effects.unique()
    b = [i.split(",") for i in a]
    z = []
    for j in b:
        for k in j:
            z.append(k)
            z = list(set(z))
            for x in z:
                if x == "None" or x=="Dry" or x=="Mouth":
                    z.remove(x)
    return z

In [4]:
filt = {"mood":{"$regex":"Happy"}}
project = {"mood":1, "product":1, "_id":0}
result = db.cbd_info.find(filt, project)
len(list(result))

1825

In [5]:
db.cbd_info.count({"mood": {"$regex": "Relaxed"}})

<ipython-input-5-b31db318e967>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  db.cbd_info.count({"mood": {"$regex": "Relaxed"}})


1693

In [18]:
def check_exists(query, collection, *pro):
    res = read_coll(collection, query, pro)
    if len(res) > 0:
        return True
    else:
        return False

In [67]:
def convert(row):
    #print(row)
    return '<a href="{}">{}</a>'.format(row['shop'],  row.name)

In [70]:
def shop_mood(moods):
    """
    This queries one collection to another and returns a DF with the 
    available CBD product given the mood chosen by user
    """
    filt = {"mood":{"$regex":f"{moods}"}}
    project = {"_id":0, "product":1}
    result = db.cbd_info.find(filt, project)
    results= list(result)
    
    lista_res = []
    for i in results:
        q = {"product":i["product"]}
        pro = {"_id":0} 
        if check_exists(q, "cbd_shops"):
            lista_res.append(read_coll("cbd_shops",q,  pro))
            flat_list = [item for sublist in lista_res for item in sublist]
            df = pd.DataFrame(flat_list)
            df["shop"] = df.apply(convert, axis=1)
            df.style.format({'shop': convert})
                    
    return df


In [74]:
def shop_mood(moods):
    filt = {"mood":{"$regex":f"{moods}"}}
    project = {"_id":0, "product":1}
    result = db.cbd_info.find(filt, project)
    results= list(result)
    
    lista_res = []
    for i in results:
        q = {"product":i["product"]}
        pro = {"_id":0} 
        if check_exists(q, "cbd_shops"):
            lista_res.append(read_coll("cbd_shops",q,  pro))
            
            lista_ratings = []
            for ratings in lista_res:
                    qu= {"product":i["product"]}
                    pro = {"_id":0}
                    if check_exists(q, "cbd_ratings"):
                           lista_ratings.append(read_coll("cbd_ratings",qu, pro))
                    


    return lista_ratings



In [75]:
shop_mood("Happy")

[[{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}],
 [{'product': 'White Widow', 'rating': 4.3}]]

In [128]:
""" STACKOVERFLOW:
def findmood():
    
db.users.aggregate([{$lookup:
  {
        from: "contacts",
        localField: "number",
        foreignField: "phonenumber",
        as: "same"
    }
   },
   {
      $match: { "same": { $ne: [] } }
   }
])

"""

' STACKOVERFLOW:\ndef findmood():\n    \ndb.users.aggregate([{$lookup:\n  {\n        from: "contacts",\n        localField: "number",\n        foreignField: "phonenumber",\n        as: "same"\n    }\n   },\n   {\n      $match: { "same": { $ne: [] } }\n   }\n])\n\n'